In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!git clone https://github.com/rezer0dai/dlppoh
#!git clone https://github.com/fgolemo/gym-ergojr
#!git clone https://github.com/mahyaret/gym-panda

#!pip install -e gym-ergojr
#!pip install -e gym-panda
#!pip install timebudget

In [3]:
#import sys
#sys.path.append("dlppoh")
#sys.path.append("gym-ergojr")
#sys.path.append("gym-panda")
#sys.path.append("/content/dlppoh")
#sys.path.append("/content/gym-ergojr")
#sys.path.append("/content/gym-panda")

In [4]:
import torch
torch.set_default_dtype(torch.float32)
torch.autograd.set_detect_anomaly(True)

In [5]:
import numpy as np
import random, timebudget

In [6]:
import config

In [7]:
from timebudget import timebudget
timebudget.set_quiet()

In [8]:
from dlppoh import *

In [9]:
from lightning import get_ready, DLPPOHLightning

/home/rezer0/.local/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [10]:
import pytorch_lightning as pl

if '__main__' == __name__:
    fm = get_ready()
    algo = DLPPOHLightning(fm)
    
    trainer = pl.Trainer(
        gpus=0,
        distributed_backend='dp',
        num_processes=1,
        max_epochs = 1000 * 100,
    )

    trainer.fit(algo)
    
    print("we are done here maybe")

/home/rezer0/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:556: UserWarning: You requested distributed training on GPUs, but none is available, so we set backend to `ddp_cpu`.
  rank_zero_warn(
/home/rezer0/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:563: UserWarning: You are running on single node with no parallelization, so distributed has no effect.
  rank_zero_warn('You are running on single node with no parallelization, so distributed has no effect.')
GPU available: False, used: False
TPU available: False, using: 0 TPU cores



LOW LEVEL POLICY: 
 [[205, '<- memory_size;', 1024, '<- batch_size;', 10, '<- optim_epochs;', 2048, '<- optim_batch_size;', 4000, '<- optim_pool_size;', 7, '<- recalc_delay;', 3, '<- sync_delta_a;', 2, '<- sync_delta_c;', 10, '<- learning_delay;', 10, '<- learning_repeat;', 0.0003, '<- lr_actor;', 0.05, '<- tau_actor;', 0.05, '<- tau_critic;', 0.2, '<- ppo_eps;', False, '<- natural;', False, '<- mean_only;']]
appended module actor_0_explorer_lowpi 0
appended module critic_0_explorer_lowpi 1
appended module critic_1_explorer_lowpi 2
appended module encoder_explorer_lowpi 3
appended module goal_encoder_explorer_lowpi 4
appended module actor_0_target_lowpi 5
appended module critic_0_target_lowpi 6
appended module encoder_target_lowpi 7
appended module goal_encoder_target_lowpi 8

HIGH LEVEL: 
 [[200, '<- memory_size;', 4096, '<- batch_size;', 10, '<- optim_epochs;', 8192, '<- optim_batch_size;', 4000, '<- optim_pool_size;', 3, '<- recalc_delay;', 3, '<- sync_delta_a;', 2, '<- sync_delta_


  | Name  | Type      | Params
------------------------------------
0 | model | FullModel | 737 K 
------------------------------------
737 K     Trainable params
135       Non-trainable params
737 K     Total params
2.949     Total estimated model params size (MB)
/home/rezer0/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Widget Javascript not detected.  It may not be installed or enabled properly.



LOW LEVEL POLICY: 
 [[205, '<- memory_size;', 1024, '<- batch_size;', 10, '<- optim_epochs;', 2048, '<- optim_batch_size;', 4000, '<- optim_pool_size;', 7, '<- recalc_delay;', 3, '<- sync_delta_a;', 2, '<- sync_delta_c;', 10, '<- learning_delay;', 10, '<- learning_repeat;', 0.0003, '<- lr_actor;', 0.05, '<- tau_actor;', 0.05, '<- tau_critic;', 0.2, '<- ppo_eps;', False, '<- natural;', False, '<- mean_only;']]

HIGH LEVEL: 
 [[200, '<- memory_size;', 4096, '<- batch_size;', 10, '<- optim_epochs;', 8192, '<- optim_batch_size;', 4000, '<- optim_pool_size;', 3, '<- recalc_delay;', 3, '<- sync_delta_a;', 2, '<- sync_delta_c;', 40, '<- learning_delay;', 80, '<- learning_repeat;', 0.0001, '<- lr_actor;', 0.07, '<- tau_actor;', 0.05, '<- tau_critic;', 0.2, '<- ppo_eps;', False, '<- natural;', False, '<- mean_only;']]
WE ARE HERE TO RESET
lot of gyms
gyms startedv4_REACHER_PANDA_hl_gym_dock_0_dlppoh_dock_15147 create REACHER_PANDA


/usr/local/lib/python3.8/dist-packages/gym-0.15.7-py3.8.egg/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


...

/usr/local/lib/python3.8/dist-packages/gym-0.15.7-py3.8.egg/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


gyms loaded


/usr/local/lib/python3.8/dist-packages/gym-0.15.7-py3.8.egg/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


TRY TO RESET
v4_REACHER_PANDA_hl_gym_dock_0_dlppoh_dock_15147 reset /w seed  0
v4_REACHER_PANDA_hl_gym_dock_1_dlppoh_dock_15147 reset /w seed  1
v4_REACHER_PANDA_hl_gym_dock_2_dlppoh_dock_15147 reset /w seed  2
v4_REACHER_PANDA_hl_gym_dock_3_dlppoh_dock_15147 reset /w seed  3
v4_REACHER_PANDA_hl_gym_dock_4_dlppoh_dock_15147 reset /w seed  4
[    1>    40:: 0] steps =   40, max_step =   0/ 40, reward=-38.000000 <action=tensor([ 0.7144, -0.3200,  0.8901])...>                    
 PUSH  5 5
WE ARE HERE TO RESET

 ep selection ---> 5
DO FINISH
DO FINISH

 PUSH  5 5
DO FINISH
TRY TO RESET
v4_REACHER_PANDA_hl_gym_dock_0_dlppoh_dock_15147 reset /w seed  100
v4_REACHER_PANDA_hl_gym_dock_1_dlppoh_dock_15147 reset /w seed  101
v4_REACHER_PANDA_hl_gym_dock_2_dlppoh_dock_15147 reset /w seed  102
v4_REACHER_PANDA_hl_gym_dock_3_dlppoh_dock_15147 reset /w seed  103
v4_REACHER_PANDA_hl_gym_dock_4_dlppoh_dock_15147 reset /w seed  104
[    2>    81:: 0] steps =   40, max_step =  41/ 40, reward=-39.00000

Process v4_REACHER_PANDA_hl_gym_dock_0_dlppoh_dock_15147:
/home/rezer0/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:895: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn('Detected KeyboardInterrupt, attempting graceful shutdown...')
Process v4_REACHER_PANDA_hl_gym_dock_1_dlppoh_dock_15147:
Process v4_REACHER_PANDA_hl_gym_dock_4_dlppoh_dock_15147:
Traceback (most recent call last):
Traceback (most recent call last):
Process v4_REACHER_PANDA_hl_gym_dock_3_dlppoh_dock_15147:
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process v4_REACHER_PANDA_hl_gym_dock_2_dlppoh_dock_15147:
Traceback (most recent call last):
  File "/home/rezer0/projects/v5opt/tasks/oaiproc.py", line 66, in run
    data = self.query.get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/rezer0/projects/v5opt/tasks/oaiproc.py", line 66, in run
    data = self.query.get()

In [11]:
for _ in env.evaluate(task, None):
    pass

NameError: name 'env' is not defined